---
authors:
  - name: Mathis Derenne
    affiliation: M2 MIASHS - Université de Lyon
    github: mathisdrn
  - name: Djida Boukari
    affiliation: M2 MIASHS - Université de Lyon
  - name: Ines Nakhli
    affiliation: M2 MIASHS - Université de Lyon
date: 2026-02-01
---

# Modèle de Gradient Boosting Survival Analysis

## Imports

In [ ]:
import numpy as np
import pandas as pd
import polars as pl
import polars.selectors as cs

## Chargement des données

In [2]:
from sksurv.util import Surv

df_train = pl.read_parquet("../../data/df_study_L18_w6_train.parquet")
df_test = pl.read_parquet("../../data/df_study_L18_w6_test.parquet")

X_train = df_train.drop(["event", "time"]).to_pandas()
X_test = df_test.drop(["event", "time"]).to_pandas()

y_train = Surv.from_dataframe("event", "time", df_train.to_pandas())
y_test = Surv.from_dataframe("event", "time", df_test.to_pandas())

## Entraînement du modèle

In [5]:
from sksurv.ensemble import GradientBoostingSurvivalAnalysis

gbs = GradientBoostingSurvivalAnalysis(
    loss="coxph", learning_rate=0.1, n_estimators=100
)
gbs.fit(X_train, y_train)

risk_gbs = gbs.predict(X_test)
surv_gbs = gbs.predict_survival_function(X_test, return_array=False)

## Évaluation du modèle final

In [6]:
from utils import evaluate_survival_model

risk_gbs = gbs.predict(X_test)
surv_gbs = gbs.predict_survival_function(X_test)
evaluate_survival_model(df_train, df_test, risk_gbs, surv_gbs)

C-index,Mean td-AUC,IBS,Brier (t=1210.0)
0.715,0.777,0.172,0.186
